# Network Engineering in the AI Era

## Module 4.2 - Introduction to Agentic AI

Welcome to this module on Agentic AI using LangGraph!
In this notebook, we'll explore what agents are, how they operate, and build a practical example of an AI agent that can help with network configuration tasks.

### What are AI Agents?

AI agents are autonomous or semi-autonomous systems that can:

- Perceive their environment through inputs and context
- Process and reason about information
- Make decisions and plan multi-step actions
- Execute tasks by using tools and APIs
- Learn from feedback and improve over time

Unlike simple LLM applications that respond to single prompts, agents maintain state, make sequential decisions, and take actions to accomplish complex tasks.

### Key Components of AI Agents

- **Memory**: Ability to store and recall information from previous interactions
- **Tools**: Functions or APIs that the agent can use to interact with external systems
- **Planning**: Ability to break down complex tasks into manageable steps
- **Reasoning**: Capability to make decisions based on available information
- **Reflection**: Ability to evaluate performance and adjust strategies

### LangGraph: Building Stateful Multi-Agent Systems

LangGraph extends LangChain by providing a framework for creating stateful agents with:

- A graph structure where nodes represent different agent states
- Tools for managing transitions between states
- Methods for orchestrating complex workflows with multiple agents

Let's start by installing the necessary libraries:

In [ ]:
!pip install -q langchain langchain-anthropic langchain-aws langchain-community langgraph langsmith boto3 rich

### Setting Up Our Environment

In order to use LangGraph, we need to setup an LLM chain. We will use ChatBedrock since our inference endpoints are hosted on Bedrock.
We will be use the Claude 3.7 model from Anthropic. This is one of the most advanced models available today.

**Note**: Use this if running from Google Collab

In [ ]:
import boto3
from langchain_aws import ChatBedrock

session = boto3.Session(
    aws_access_key_id="your-key-id",
    aws_secret_access_key="your-key",
    region_name="us-east-1"
)

llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_kwargs=dict(temperature=0),
    client=session.client("bedrock-runtime")
  )

**Note**: Use this if running from AWS directly

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0,
    region="us-east-1",
)

### Building Our Agent's Components

Our graph will have a single agent (the assistant) and some available tools.

Let's start by defining the tools, and begin with our **Policy Lookup Tool**.

This tool will take the markdown document `acme_guidelines.md` and will split it using the MarkdownHeaderTextSplitter (the same one we learned about on our previous module) and will store the resulting documents in a `InMemoryVectorStore`.
This `InMemoryVectorStore` is no different than any other vector storage databases, it won't be used in a production environment but it is useful for training purposes.

The lookup_policy tool once called, will do a similarity search on the documents stored in the `InMemoryVectorStore` and will return the most similar document to the query.

**Note**: Use this if running from Google Collab

In [ ]:
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0", 
    client=session.client("bedrock-runtime")
)

**Note**: Use this if running from AWS directly

In [ ]:
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0", 
    region_name='us-east-1'
)

In [ ]:
from typing import Annotated
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.document_loaders import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter

loader = TextLoader("acme_guidelines.md")
documents = loader.load()
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_splits = []
for document in documents:
    md_splits.extend(md_splitter.split_text(document.page_content))

vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(md_splits)


@tool
def lookup_policy(
    query: Annotated[str, "The query to look for in the company policies."],
) -> str:
    """
    Consult the configuration guidelines.
    Use this before making any network changes performing other 'write' events.
    """
    docs = vector_store.similarity_search(query, k=2)
    return "\n\n".join([doc.page_content for doc in docs])

Now that we have done that, let's proceed with creating a tool that will allow the agent to configure devices. In this example, our tool will simulate sending the configuration to a device, and will instead print the configuration to the console. In a real-world scenario, you would replace this with actual code that sends the configuration to a device for example via RESTCONF or NETCONF.

**Note**: Please select the port number assigned to you. Update the `hostname` as well

In [ ]:
from typing import Annotated
from langchain_core.tools import tool
from netmiko import ConnectHandler

devices = {
    "CTG-EDGE-001": {
        "hostname": "54.81.253.219", # Update IP based on lab details
        "username": "admin",
        "password": "cisco123",
        "port": 8022, # Update the port accordingly
    }
}

@tool
def add_device_interface(
    device_name: Annotated[str, "The name of the device to configure."],
    interface_name: Annotated[
        str,
        "The interface name to apply the configuration to. E.g., 'Loopback100'",
    ],
    interface_description: Annotated[str, "The description to set for the interface."],
    interface_ipv4_address: Annotated[
        str, "The IPv4 address to assign to the interface. E.g., 192.168.1.1"
    ],
    interface_ipv4_subnet_mask: Annotated[
        str, "The IPv4 subnet mask to assign to the interface. E.g., 255.255.255.0"
    ],
) -> str:
    """Use this tool to add an interface to a device."""

    connection = {
        'device_type': 'cisco_xr',
        'ip': devices[device_name]["hostname"],
        'username': devices[device_name]["username"],
        'password': devices[device_name]["password"],
        'port': devices[device_name]["port"],
        'secret': '',
        'verbose': False
    }

    config_commands = [
        f'interface {interface_name}',
        f'description {interface_description}',
        f'ipv4 address {interface_ipv4_address} {interface_ipv4_subnet_mask}',
        'no shutdown'
    ]

    try:
        net_connect = ConnectHandler(**connection)
        net_connect.send_config_set(config_commands)
        net_connect.send_command('commit')
    except Exception as e:
        return f"Exception {e}"

    net_connect.disconnect()

    return f"Interface {interface_name} successfully configured"
   


@tool
def get_device_interface(
    device_name: Annotated[str, "The name of the device to configure."],
    interface_name: Annotated[
        str,
        "The interface name to apply the configuration to. E.g., 'Loopback100'",
    ],
) -> str:
    """Use this tool to get the configuration for an interface of a device."""

    connection = {
        'device_type': 'cisco_xr',
        'ip': devices[device_name]["hostname"],
        'username': devices[device_name]["username"],
        'password': devices[device_name]["password"],
        'port': devices[device_name]["port"],
        'secret': '',
        'verbose': False
    }

    net_connect = ConnectHandler(**connection)
    output = net_connect.send_command(f"show interface {interface_name}")
    net_connect.disconnect()
    return output


Let's create an array that will contain the list of tools that will be available to our agent. The first tool we'll define is the policy lookup tool. This tool will allow the agent to look up network policies in the knowledge base.

In [ ]:
from langchain_core.runnables import RunnableLambda


available_tools = [
    lookup_policy,
    add_device_interface,
    get_device_interface,
]

Let's create our assistant agent that will have access to all the available tools. We'll define the agent as a class. This will allow us to easily add more tools in the future if needed.

In [ ]:
from datetime import datetime

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable, RunnableConfig
from langgraph.graph import MessagesState

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "You are a helpful customer support assistant for ACME Service Provider. "
            "Use the provided tools to create resources or to search for procedures, company policies, and other information to assist the user's queries. "
            "When searching, be persistent. Expand your query bounds if the first search returns no results. "
            "If a search comes up empty, expand your search before giving up."
            "\nCurrent time: {time}.",
        ),
        MessagesPlaceholder("messages"),
    ]
).partial(time=datetime.now)


class Assistant:
    """Assistant node for the graph."""

    def __init__(self):
        """Initialize the assistant."""
        self.runnable: Runnable = primary_assistant_prompt | llm.bind_tools(
            available_tools
        )

    def __call__(self, state: MessagesState, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [
                    HumanMessage(content="Respond with a real output.")
                ]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}

Building the Agent Graph
Now, let's connect all these components into a graph using LangGraph:

In [ ]:
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import tools_condition
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode

# Create the graph
workflow = StateGraph(MessagesState)

# Add nodes: these do the work
workflow.add_node("assistant", Assistant())
workflow.add_node(
    "tools",
    ToolNode(available_tools),
)

# Define edges: these determine how the control flow moves
workflow.add_edge(START, "assistant")
workflow.add_conditional_edges(
    "assistant",
    tools_condition,
)
workflow.add_edge("tools", "assistant")

graph = workflow.compile()


Let's visualize our graph to see how the nodes are connected:

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

Using Our Network Configuration Agent
Let's create a function to interact with our agent:

Example Usage
Let's test our agent with a few examples. Let's begin with asking our agent the following: **I need to create a new VLAN for my database server.**

In [ ]:
import rich
from langchain_core.messages import HumanMessage

initial_message = HumanMessage(
    content="Configure interface Loopback101 on device CTG-EDGE-001 with IPv4 address 10.1.2.2/30, this interface will connect to BOG-CORE-001."
)
rich.print(initial_message.pretty_repr())
for event in graph.stream(
    {"messages": [initial_message]},
    stream_mode="updates",
):
    for _, message_or_messages in event.items():
        if isinstance(message_or_messages["messages"], list):
            for message in message_or_messages["messages"]:
                rich.print(message.pretty_repr())
        else:
            rich.print(message_or_messages["messages"].pretty_repr())


Let's now verify that the requested interface has been configured.

In [ ]:
initial_message = HumanMessage(
    content="In device CTG-EDGE-001, which IP is configured on interface Loopback101?"
)
rich.print(initial_message.pretty_repr())
for event in graph.stream(
    {"messages": [initial_message]},
    stream_mode="updates",
):
    for _, message_or_messages in event.items():
        if isinstance(message_or_messages["messages"], list):
            for message in message_or_messages["messages"]:
                rich.print(message.pretty_repr())
        else:
            rich.print(message_or_messages["messages"].pretty_repr())